In [1]:
import numpy as np
import pandas as pd
import re
import string
import math
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from nltk.stem import WordNetLemmatizer


dataFile = 'data/spam.csv'
trainRatio = 80

data = pd.read_csv(dataFile, usecols=['v1','v2'], encoding='ISO-8859-1')
data.rename(columns={'v1':'label'}, inplace=True)
data.rename(columns={'v2':'text'}, inplace=True)

In [2]:
# text preprocessing
def preprocess(data):


    # replace email addresses with emailaddress
    data['text'] = data['text'].apply(lambda x: re.sub('([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', 'emailaddress',x))

    # replace website links with websiteaddress
    data['text'] = data['text'].apply(lambda x:re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'websiteaddress', x))

    # expand contractions
    data['text'] = data['text'].apply(lambda x: re.sub(r"i'm", "i am", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"he's", "he is", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"she's", "she is", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"that's", "that is", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"what's", "what is", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"where's", "where is", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"\'ll", " will", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"\'ve", " have", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"\'re", " are", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"\'d", " would", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"\'ve", " have", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"won't", "will not", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"don't", "do not", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"did't", "did not", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"can't", "can not", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"it's", "it is", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"couldn't", "could not", x))
    data['text'] = data['text'].apply(lambda x: re.sub(r"have't", "have not", x))

    # remove punctuations except !$+():-;&
    data['text'] = data['text'].apply(lambda x: re.sub(r"[,.\"@#%^*{}?/`'~<>=]", "", x))

    # remove stop words
    stop_words = set(stopwords.words('english'))
    stop_words.discard("not")

    def remove_stopwords(text):
        return " ".join([word for word in str(text).split() if word not in stop_words])
    data['text'] = data['text'].apply(lambda x: remove_stopwords(x))


    return data



# feature extraction
# 0  capital words frequency
# 1  ! frequency
# 2  £ frequency
# 3  + frequency
# 4  slimies frequency like :) :( :-) :-( ;-) ;-( :-D
# 5  prize frequency
# 6  credit frequency
# 7  urgent frequency
# 8  free frequency
# 9  auction frequency
# 10 guaranteed frequency
# 11 contains numbers
# 12 contains numbers with 00
# 13 contains weblinks
# 14 contains email addresses
# 15 amp frequency
# 16 &gt; and &lt; frequency
# 17 lol frequency
# 18 alphanumeric words frequency

def extractFeatures(data):
    features = np.zeros((len(data['text']), 19))

    texts = data['text']

    # 0
    for i, text in enumerate(texts):
        features[i][0] = len(re.findall(r"(\b(?:[A-Z]+[a-z]?[A-Z]*|[A-Z]*[a-z]?[A-Z]+)\b(?:\s+(?:[A-Z]+[a-z]?[A-Z]*|[A-Z]*[a-z]?[A-Z]+)\b)*)", text))

    data = preprocess(data)

    texts = data['text']

    for i, text in enumerate(texts):
        features[i][1] = text.count('!')
        features[i][2] = text.count('£')

        features[i][3] = text.count('+')

        features[i][4] += text.count(':)')
        features[i][4] += text.count(':(')
        features[i][4] += text.count(':-)')
        features[i][4] += text.count(':-(')
        features[i][4] += text.count(';-)')
        features[i][4] += text.count(';-(')
        features[i][4] += text.count(':-D')

        features[i][5] = text.count('prize')
        features[i][6] = text.count('credit')
        features[i][7] = text.count('urgent')
        features[i][8] = text.count('free')
        features[i][9] = text.count('auction')
        features[i][10] = text.count('guaranteed')

        features[i][11] = len(re.findall(r'\d', text))
        features[i][12] = 1 if text.count('00') > 0 else 0

        features[i][13] = 1 if text.count('websiteaddress') or text.count('http') > 0 else 0
        features[i][14] = 1 if text.count('emailaddress') > 0 else 0

        features[i][15] = text.count('amp')

        features[i][16] = text.count('&gt;')
        features[i][17] += text.count('&lt;')

        features[i][18] = len(re.findall(r'(?:\d+[a-zA-Z]+|[a-zA-Z]+\d+)', text))

    return features

In [3]:
nltk.download('stopwords')
features = extractFeatures(data)


x = features
y = data['label'].values

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
dataCount = len(x)
trainCount = math.floor(dataCount * trainRatio / 100)
xTrain, yTrain = x[0:trainCount], y[0:trainCount]
xTest, yTest = x[trainCount:], y[trainCount:]

model = GaussianNB()
model.fit(xTrain, yTrain)

GaussianNB()

In [5]:
tp, fp, tn, fn = 0, 0, 0, 0

for i, testSample in enumerate(xTest):
    yPredicted = model.predict([testSample])

    if yTest[i] == 'ham':
        if yPredicted == yTest[i]:
            tp += 1
        else:
            fn += 1
    else:
        if yPredicted == yTest[i]:
            tn += 1
        else:
            fp += 1


accuracy = (tp + tn) / (tp + tn + fp + fn)
recall = (tp) / (tp + fn)
precision = (tp) / (tp + fp)
f1 = (2 * tp) / ((2 * tp) + (fp + fn))


print('Total Samples: ' + str(len(xTest)))
print('TP: ' + str(tp))
print('TN: ' + str(tn))
print('FP: ' + str(fp))
print('FN: ' + str(fn))
print('Accuracy: ' + str(accuracy))
print('Recall: ' + str(recall))
print('Precision: ' + str(precision))
print('F1: ' + str(f1))


Total Samples: 1115
TP: 949
TN: 127
FP: 18
FN: 21
Accuracy: 0.9650224215246637
Recall: 0.9783505154639175
Precision: 0.9813857290589452
F1: 0.9798657718120806
